<a href="https://colab.research.google.com/github/gkpatil311/deploying-machine-learning-models/blob/master/01_predicting_survival_titanic_assignement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Predicting Survival on the Titanic

### History
Perhaps one of the most infamous shipwrecks in history, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 people on board. Interestingly, by analysing the probability of survival based on few attributes like gender, age, and social status, we can make very accurate predictions on which passengers would survive. Some groups of people were more likely to survive than others, such as women, children, and the upper-class. Therefore, we can learn about the society priorities and privileges at the time.

### Assignment:

Build a Machine Learning Pipeline, to engineer the features in the data set and predict who is more likely to Survive the catastrophe.

Follow the Jupyter notebook below, and complete the missing bits of code, to achieve each one of the pipeline steps.

In [206]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

## Prepare the data set

In [207]:
# load the data - it is available open source and online

data = pd.read_csv('https://www.openml.org/data/get_csv/16826755/phpMYEkMl')

# display data
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"


In [208]:
# replace interrogation marks by NaN values

data = data.replace('?', np.nan)

In [209]:
data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,NaN,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


In [210]:
# retain only the first cabin if more than
# 1 are available per passenger

def get_first_cabin(row):
    try:
        # print(row.split())
        return row.split()[0]
    except:
        return np.nan
    
data['cabin'] = data['cabin'].apply(get_first_cabin)

In [211]:
# extracts the title (Mr, Ms, etc) from the name variable

def get_title(passenger):
    line = passenger
    if re.search('Mrs', line):
        return 'Mrs'
    elif re.search('Mr', line):
        return 'Mr'
    elif re.search('Miss', line):
        return 'Miss'
    elif re.search('Master', line):
        return 'Master'
    else:
        return 'Other'
    
data['title'] = data['name'].apply(get_title)

In [212]:
# cast numerical variables as floats

data['fare'] = data['fare'].astype('float')
data['age'] = data['age'].astype('float')

In [213]:
# drop unnecessary variables

data.drop(labels=['name','ticket', 'boat', 'body','home.dest'], axis=1, inplace=True)

# display data
data.head()

,pclass,survived,sex,age,sibsp,parch,fare,cabin,embarked,title
0,1,1,female,29.0000,0,0,211.3375,B5,S,Miss
1,1,1,male,0.9167,1,2,151.5500,C22,S,Master
2,1,0,female,2.0000,1,2,151.5500,C22,S,Miss
3,1,0,male,30.0000,1,2,151.5500,C22,S,Mr
4,1,0,female,25.0000,1,2,151.5500,C22,S,Mrs


In [214]:
# save the data set

data.to_csv('titanic.csv', index=False)

## Data Exploration

### Find numerical and categorical variables

In [215]:
target = 'survived'

In [216]:

vars_num = [var for var in data.columns if data[var].dtypes != "O"]# fill your code here
 
vars_cat = [var for var in data.columns if data[var].dtypes == "O"]# fill your code here

print('Number of numerical variables: {}'.format(len(vars_num)))
print('Number of categorical variables: {}'.format(len(vars_cat)))

Number of numerical variables: 6
Number of categorical variables: 4


### Find missing values in variables

In [217]:
# first in numerical variables
vars_numeric_na = [var for var in vars_num if data[var].isnull().sum()>0]
print(vars_numeric_na)


['age', 'fare']


In [218]:
# now in categorical variables
vars_categorical_na = [var for var in vars_cat if data[var].isnull().sum()>0]
print(vars_categorical_na)


['cabin', 'embarked']


### Determine cardinality of categorical variables

In [219]:
data[vars_cat].nunique()

sex           2
cabin       181
embarked      3
title         5
dtype: int64

### Determine the distribution of numerical variables

In [220]:
data[vars_num].nunique()

pclass        3
survived      2
age          98
sibsp         7
parch         8
fare        281
dtype: int64

## Separate data into train and test

Use the code below for reproducibility. Don't change it.

In [221]:
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('survived', axis=1),  # predictors
    data['survived'],  # target
    test_size=0.2,  # percentage of obs in test set
    random_state=0)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((1047, 9), (262, 9))

## Feature Engineering

### Extract only the letter (and drop the number) from the variable Cabin

In [222]:
# print(data.cabin)
# cabin_first = [var for var in X_train.cabin if var.notnull()]
X_train["cabin"] = np.where(X_train["cabin"].notnull(),X_train["cabin"].astype(str).str[0],np.nan)
X_test["cabin"] = np.where(X_test["cabin"].notnull(),X_test["cabin"].astype(str).str[0],np.nan)
 

In [223]:
X_train["cabin"].unique()

array([nan, 'E', 'F', 'A', 'C', 'D', 'B', 'T', 'G'], dtype=object)

### Fill in Missing data in numerical variables:

- Add a binary missing indicator
- Fill NA in original variable with the median

In [224]:
# print(X_train[vars_numeric_na].isnull().sum())
for var in vars_numeric_na:
  median_col = X_train[var].median()
  print(var, "median:",median_col)
  X_train[var+"_na"] =  np.where(X_train[var].isnull(),1,0)
  X_test[var+"_na"] =  np.where(X_test[var].isnull(),1,0)

  X_train[var] =   X_train[var].fillna(median_col)
  X_test[var] =   X_test[var].fillna(median_col)

# print(X_train[vars_numeric_na].isnull().sum())  

age median: 28.0
fare median: 14.4542


### Replace Missing data in categorical variables with the string **Missing**

In [225]:
X_train[vars_categorical_na] =   X_train[vars_categorical_na].fillna("Missing")
X_test[vars_categorical_na] =   X_test[vars_categorical_na].fillna("Missing")

# X_train[vars_categorical_na] = X_train[vars_categorical_na].fillna('Missing')
# X_test[vars_categorical_na] = X_test[vars_categorical_na].fillna('Missing')

In [226]:
print(X_train[vars_categorical_na].isnull().sum())  
print(X_test[vars_categorical_na].isnull().sum())  

cabin       0
embarked    0
dtype: int64
cabin       0
embarked    0
dtype: int64


### Remove rare labels in categorical variables

- remove labels present in less than 5 % of the passengers

In [227]:
def find_frequent_labels(df,var):
  df= df.copy()
  tmp = df.groupby(var)[var].count()/len(df)
  # print(tmp)
  return tmp[tmp > 0.05].index

for var in vars_cat:
  frequent_cat = find_frequent_labels(X_train , var)
  print(frequent_cat)
  X_train[var] = np.where(X_train[var].isin(frequent_cat),X_train[var],"Rare")
  X_test[var] = np.where(X_test[var].isin(frequent_cat),X_test[var],"Rare")

Index(['female', 'male'], dtype='object', name='sex')
Index(['C', 'Missing'], dtype='object', name='cabin')
Index(['C', 'Q', 'S'], dtype='object', name='embarked')
Index(['Miss', 'Mr', 'Mrs'], dtype='object', name='title')


In [228]:
X_train["cabin"].value_counts()

Missing    803
Rare       166
C           78
Name: cabin, dtype: int64

In [229]:
X_test["cabin"].value_counts()

Missing    211
Rare        35
C           16
Name: cabin, dtype: int64

### Perform one hot encoding of categorical variables into k-1 binary variables

- k-1, means that if the variable contains 9 different categories, we create 8 different binary variables
- Remember to drop the original categorical variable (the one with the strings) after the encoding

In [230]:
# X_train[vars_cat].isnull().sum()
# X_train[vars_cat].value_counts()

In [231]:

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

ct = ColumnTransformer(transformers=[('OHEncode',OneHotEncoder(drop='first'),vars_cat)],remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.transform(X_test))

In [232]:
# ohe_data_train = pd.get_dummies(X_train,columns=vars_cat , drop_first =True)
# ohe_data_test = pd.get_dummies(X_test,columns=vars_cat, drop_first =True)

In [233]:
# print(ohe_data_train.shape,ohe_data_test.shape)
# list_train_col = ohe_data_train.columns
# # print(list_train_col.drop(ohe_data_test.columns))
# ohe_data_test[list_train_col.drop(ohe_data_test.columns)] =  0
# print(ohe_data_train.shape,ohe_data_test.shape)
# print(X_train.shape,X_test.shape)
# X_train = pd.concat([ohe_data_train])
# X_test = pd.concat([ohe_data_test])
# print(X_train.shape,X_test.shape)

### Scale the variables

- Use the standard scaler from Scikit-learn

In [234]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
#  fit  the scaler to the train set
scaler.fit(X_train)

# transform the train and test set
X_train = scaler.transform(X_train)

X_test = scaler.transform(X_test)

In [235]:
X_train

array([[ 0.76104239,  0.55123545, -0.43407632, ...,  0.86989161,
        -0.43562912, -0.27577789],
       [-1.31398725, -1.81410683,  2.30374237, ..., -1.14956851,
        -0.43562912, -0.27577789],
       [ 0.76104239,  0.55123545, -0.43407632, ...,  0.86989161,
        -0.43562912, -0.27577789],
       ...,
       [-1.31398725,  0.55123545, -0.43407632, ..., -1.14956851,
        -0.43562912, -0.27577789],
       [-1.31398725,  0.55123545, -0.43407632, ..., -1.14956851,
        -0.43562912, -0.27577789],
       [-1.31398725,  0.55123545, -0.43407632, ..., -1.14956851,
         2.29553067, -0.27577789]])

## Train the Logistic Regression model

- Set the regularization parameter to 0.0005
- Set the seed to 0

In [236]:
log_reg = LogisticRegression(penalty='l1', tol=0.0005, random_state=0, solver='liblinear', l1_ratio=1)
log_reg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:1479: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  "(penalty={})".format(self.penalty)


LogisticRegression(l1_ratio=1, penalty='l1', random_state=0, solver='liblinear',
                   tol=0.0005)

## Make predictions and evaluate model performance

Determine:
- roc-auc
- accuracy

**Important, remember that to determine the accuracy, you need the outcome 0, 1, referring to survived or not. But to determine the roc-auc you need the probability of survival.**

In [237]:
y_pred = log_reg.predict(X_test)

print('Accuracy: %.3f' % accuracy_score(y_test.values, y_pred))
print('ROC-AUC : %.3f' % roc_auc_score(y_test.values, y_pred))

Accuracy: 0.763
ROC-AUC : 0.742


That's it! Well done

**Keep this code safe, as we will use this notebook later on, to build production code, in our next assignement!!**